## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import gmaps
import requests
from config import weather_api_key
from config import g_key
import time
from datetime import datetime
from scipy.stats import linregress

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/Users/Ryan/Desktop/Data_classwork/World_Weather_Analysis/Weather_database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Ponta Do Sol,PT,2022-05-15 23:43:35,32.6667,-17.1000,69.71,82,9,6.06
1,1,Kontagora,NG,2022-05-15 23:43:36,10.3999,5.4695,82.89,53,90,3.62
2,2,Husavik,IS,2022-05-15 23:43:37,66.0449,-17.3389,46.18,86,63,9.28
3,3,La Ronge,CA,2022-05-15 23:40:53,55.1001,-105.2842,48.42,49,75,11.50
4,4,East London,ZA,2022-05-15 23:43:37,-33.0153,27.9116,61.84,85,0,8.70


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Kontagora,NG,2022-05-15 23:43:36,10.3999,5.4695,82.89,53,90,3.62
5,5,Nouadhibou,MR,2022-05-15 23:43:38,20.9310,-17.0347,75.18,64,94,17.27
6,6,Bambous Virieux,MU,2022-05-15 23:43:15,-20.3428,57.7575,75.45,83,40,11.50
11,11,Hilo,US,2022-05-15 23:41:56,19.7297,-155.0900,85.28,74,75,12.66
23,23,Rikitea,PF,2022-05-15 23:43:48,-23.1203,-134.9692,76.73,68,39,18.21
27,27,Atuona,PF,2022-05-15 23:43:51,-9.8000,-139.0333,78.67,74,15,16.80
31,31,Sulya,IN,2022-05-15 23:43:54,12.5667,75.3833,76.12,98,100,1.19
32,32,Roma,US,2022-05-15 23:43:55,43.2128,-75.4557,78.94,53,0,8.05
37,37,San Cristobal,VE,2022-05-15 23:43:57,7.7669,-72.2250,77.18,92,100,2.93
53,53,Kavaratti,IN,2022-05-15 23:45:07,10.5669,72.6420,81.46,80,100,20.85


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       0
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Kontagora,NG,2022-05-15 23:43:36,10.3999,5.4695,82.89,53,90,3.62
5,5,Nouadhibou,MR,2022-05-15 23:43:38,20.9310,-17.0347,75.18,64,94,17.27
6,6,Bambous Virieux,MU,2022-05-15 23:43:15,-20.3428,57.7575,75.45,83,40,11.50
11,11,Hilo,US,2022-05-15 23:41:56,19.7297,-155.0900,85.28,74,75,12.66
23,23,Rikitea,PF,2022-05-15 23:43:48,-23.1203,-134.9692,76.73,68,39,18.21
...,...,...,...,...,...,...,...,...,...,...
679,679,Cabadiangan,PH,2022-05-16 00:03:46,9.7531,122.4739,83.35,73,97,2.06
682,682,Ibra,OM,2022-05-16 00:03:48,22.6906,58.5334,75.00,35,0,3.09
686,686,Estelle,US,2022-05-16 00:03:49,29.8458,-90.1067,87.01,61,0,4.61
691,691,Ahuimanu,US,2022-05-16 00:03:51,21.4447,-157.8378,80.37,67,0,12.66


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Kontagora,NG,82.89,10.3999,5.4695,
5,Nouadhibou,MR,75.18,20.9310,-17.0347,
6,Bambous Virieux,MU,75.45,-20.3428,57.7575,
11,Hilo,US,85.28,19.7297,-155.0900,
23,Rikitea,PF,76.73,-23.1203,-134.9692,
27,Atuona,PF,78.67,-9.8000,-139.0333,
31,Sulya,IN,76.12,12.5667,75.3833,
32,Roma,US,78.94,43.2128,-75.4557,
37,San Cristobal,VE,77.18,7.7669,-72.2250,
53,Kavaratti,IN,81.46,10.5669,72.6420,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print('Hotel not found... skipping.')
hotel_df.head(10)
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Kontagora,NG,82.89,10.3999,5.4695,Queens Guest Inn
5,Nouadhibou,MR,75.18,20.9310,-17.0347,El Medina
6,Bambous Virieux,MU,75.45,-20.3428,57.7575,Casa Tia Villa
11,Hilo,US,85.28,19.7297,-155.0900,Hilo Hawaiian Hotel
23,Rikitea,PF,76.73,-23.1203,-134.9692,People ThankYou
27,Atuona,PF,78.67,-9.8000,-139.0333,Villa Enata
31,Sulya,IN,76.12,12.5667,75.3833,RIVER ISLAND LODGING
32,Roma,US,78.94,43.2128,-75.4557,Wingate By Wyndham Rome
37,San Cristobal,VE,77.18,7.7669,-72.2250,Pirineos
53,Kavaratti,IN,81.46,10.5669,72.6420,Hotels in Lakshadweep Islands


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()

City          0
Country       0
Max Temp      0
Lat           0
Lng           0
Hotel Name    0
dtype: int64

In [14]:
# 8a. Create the output File (CSV)
output_data_file = "C:/Users/Ryan/Desktop/Data_classwork/World_Weather_Analysis/Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [17]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Kontagora,NG,82.89,10.3999,5.4695,Queens Guest Inn
5,Nouadhibou,MR,75.18,20.9310,-17.0347,El Medina
6,Bambous Virieux,MU,75.45,-20.3428,57.7575,Casa Tia Villa
11,Hilo,US,85.28,19.7297,-155.0900,Hilo Hawaiian Hotel
23,Rikitea,PF,76.73,-23.1203,-134.9692,People ThankYou
27,Atuona,PF,78.67,-9.8000,-139.0333,Villa Enata
31,Sulya,IN,76.12,12.5667,75.3833,RIVER ISLAND LODGING
32,Roma,US,78.94,43.2128,-75.4557,Wingate By Wyndham Rome
37,San Cristobal,VE,77.18,7.7669,-72.2250,Pirineos
53,Kavaratti,IN,81.46,10.5669,72.6420,Hotels in Lakshadweep Islands


In [18]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))